<a href="https://colab.research.google.com/github/rayyirahmaid/Clothing-Classification-Model-with-CNN-and-Transformer/blob/main/Tubes_18321013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import timm

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, KFold
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, datasets, models
from torchvision.models import alexnet
from google.colab import drive

In [ ]:
# Mount Drive ke memori google collab
drive.mount('/content/drive')

# Path ke dataset utama
dataset_path = '/content/drive/My Drive/processed_dataset'  # Ganti dengan path dataset utama Anda

# Path ke folder train dan test di /content
train_dir = "/content/train"  # Path untuk training
test_dir = "/content/test"  # Path untuk testing

# Membuat folder Train dan Test
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Step 4: Loop untuk setiap kelas dalam folder Dataset
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):  # Pastikan hanya memproses folder kelas
        print(f"Memproses kelas: {class_name}")

        # Buat folder untuk kelas ini di Train dan Test
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

        # Ambil semua file dari kelas
        all_files = [os.path.join(class_path, f) for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

        # Bagi file ke training (70%) dan testing (30%) secara acak
        train_files, test_files = train_test_split(all_files, test_size=0.3, random_state=42)

        # Pindahkan file ke folder Train dan Test
        for file in train_files:
            shutil.copy(file, os.path.join(train_dir, class_name))
        for file in test_files:
            shutil.copy(file, os.path.join(test_dir, class_name))

print("Dataset telah berhasil dibagi menjadi Train dan Test!")
print(f"Folder Train: {train_dir}")
print(f"Folder Test: {test_dir}")

In [ ]:
# Load Dataset
batch_size = 32
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Load Model
num_classes = len(train_dataset.classes)
model = alexnet(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20

In [ ]:
# Training Loop
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    train_loss = running_loss / len(train_loader)
    train_acc = correct / total
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)

    # Validation Loop
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(outputs.argmax(1).cpu().numpy())
    val_loss = running_loss / len(test_loader)
    val_acc = correct / total
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

In [ ]:
# Menyimpan model ke file
model_path = "/content/drive/My Drive/saved_model.pth"  # Ganti dengan path penyimpanan Anda
torch.save(model.state_dict(), model_path)
print(f"Model telah disimpan ke {model_path}")

In [ ]:
# Evaluation and Metrics
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=train_dataset.classes))
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_dataset.classes)
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.show()

In [ ]:
# Import Dataset
train_dir = '/content/train'  # Make sure this points to your training data
model_save_path = '/content/vit_clothing_model.pth'
batch_size = 32
num_epochs = 20  # Start with 5-10 epochs to see how it performs
learning_rate = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Training on device: {device}")

In [ ]:
# Data Preparation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

print(f"Loading training data from: {train_dir}")
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

class_names = train_dataset.classes
num_classes = len(class_names)
print(f"Training on {num_classes} classes: {class_names}")

In [ ]:
# Model Initialization
print("Downloading Pre-trained Vision Transformer...")
# pretrained=True downloads the weights. num_classes replaces the output layer.
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
model = model.to(device)

# Define how we measure error (CrossEntropy) and how we update weights (Adam optimizer)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# The Training Loop
print("Starting training process...")
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass: predict the clothing
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass: calculate the errors and update weights
        loss.backward()
        optimizer.step()

        # Track accuracy and loss
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc*100:.2f}%")

# Save the Fine-Tuned Model
torch.save(model.state_dict(), model_save_path)
print(f"Training complete! Model saved successfully to {model_save_path}")

Starting training process...


In [ ]:
# Testing Dataset
test_dir = '/content/test'
model_weights_path = '/content/vit_clothing_model.pth' # Pointing to the file we just saved!
batch_size = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Evaluating on device: {device}")

# Data Preparation (No Augmentation)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
class_names = test_dataset.classes
num_classes = len(class_names)

# 3. Model Initialization
print("Initializing Vision Transformer...")
# pretrained=False because we are about to load our own custom weights
model = timm.create_model('vit_base_patch16_224', pretrained=False, num_classes=num_classes)

# Load the weights we just trained
if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path, map_location=device))
    print("Custom weights loaded successfully!")
else:
    print(f"Error: Could not find weights at {model_weights_path}. Did you run the training script first?")

model = model.to(device)
model.eval() # Set to evaluation mode

# Evaluation Loop
y_true = []
y_pred = []

print("Running predictions on test data...")
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Metrics and Visualization
print("\n" + "="*50)
print("Classification Report:")
print("="*50)
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

fig, ax = plt.subplots(figsize=(10, 8))
disp.plot(cmap=plt.cm.Blues, values_format='d', ax=ax)
plt.title('Vision Transformer Confusion Matrix')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()